In [1]:
# Adjust Python path within the notebook
import sys
project_root = '/Users/thangnguyen/Documents/GitHub/project-1-individual-knam2609'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import scripts

In [2]:
from pyspark.sql.functions import to_date, col

In [3]:
# Download weather data
scripts.download.retrieve_file("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/retrievebulkdataset?&key=U5YR6UTVW5E3RR7RVGSSUZVGL&taskId=74b466562ef06ab22e9d288f264aed81&zip=false", "../data/landing/weather.csv")

In [4]:
# Create SparkSession
spark = scripts.clean_base.create_spark_session()

24/08/27 18:58:30 WARN Utils: Your hostname, THANGs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 100.86.89.69 instead (on interface en0)
24/08/27 18:58:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/27 18:58:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/27 18:58:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
weather_df = spark.read.csv("../data/landing/weather.csv", header=True, inferSchema=True)
weather_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- datetime: date (nullable = true)
 |-- tempmax: double (nullable = true)
 |-- tempmin: double (nullable = true)
 |-- temp: double (nullable = true)
 |-- feelslikemax: double (nullable = true)
 |-- feelslikemin: double (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- dew: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- precip: double (nullable = true)
 |-- precipprob: integer (nullable = true)
 |-- precipcover: double (nullable = true)
 |-- preciptype: string (nullable = true)
 |-- snow: integer (nullable = true)
 |-- snowdepth: integer (nullable = true)
 |-- windgust: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- winddir: double (nullable = true)
 |-- sealevelpressure: double (nullable = true)
 |-- cloudcover: double (nullable = true)
 |-- visibility: double (nullable = true)
 |-- solarradiation: double (nullable = true)
 |-- solarenergy: double (nullable = true)
 |-- uvindex: int

In [6]:
weather_df = weather_df.select(["datetime", "temp", "humidity", "precip"])

In [7]:
weather_df

datetime,temp,humidity,precip
2023-06-01,20.9,64.4,0.23
2023-06-02,22.9,60.0,0.298
2023-06-03,18.8,68.7,0.0
2023-06-04,16.0,58.2,0.23
2023-06-05,19.2,50.3,0.23
2023-06-06,20.7,43.1,0.019
2023-06-07,18.4,46.2,0.0
2023-06-08,18.3,51.5,0.0
2023-06-09,18.4,57.4,0.0
2023-06-10,20.2,55.4,0.23


In [8]:
weather_attrs = ["temp", "humidity", "precip"]

In [9]:
yellow_df = spark.read.parquet("../data/curated/yellow/yellow.parquet")
yellow_df = yellow_df.withColumn("datetime", to_date(col("pickup_datetime")))
yellow_with_weather_df = yellow_df.join(weather_df, on="datetime", how="inner")
yellow_with_weather_df = yellow_with_weather_df.drop("datetime")
yellow_with_weather_df

VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_time,fare_per_miles,temp,humidity,precip
2,2023-06-01 00:11:50,2023-06-01 00:15:47,1,0.99,1,186,234,1,6.5,1.0,0.5,1.72,0.0,1.0,13.22,2.5,0.0,3.95,13.353535353535355,20.9,64.4,0.23
2,2023-06-01 00:31:14,2023-06-01 00:36:04,1,0.98,1,249,125,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.0,4.833333333333333,14.938775510204083,20.9,64.4,0.23
2,2023-06-01 00:39:29,2023-06-01 00:44:49,1,1.42,1,137,229,1,8.6,1.0,0.5,2.4,0.0,1.0,16.0,2.5,0.0,5.333333333333333,11.267605633802818,20.9,64.4,0.23
2,2023-06-01 00:41:56,2023-06-01 00:48:35,1,1.62,1,162,100,1,10.0,1.0,0.5,3.75,0.0,1.0,18.75,2.5,0.0,6.65,11.574074074074073,20.9,64.4,0.23
2,2023-06-01 00:06:27,2023-06-01 00:14:32,1,2.23,1,236,229,1,11.4,1.0,0.5,1.6,0.0,1.0,18.0,2.5,0.0,8.083333333333334,8.071748878923767,20.9,64.4,0.23
1,2023-06-01 00:12:23,2023-06-01 00:24:54,1,2.7,1,141,170,1,14.9,3.5,0.5,3.95,0.0,1.0,23.85,2.5,0.0,12.516666666666667,8.833333333333334,20.9,64.4,0.23
2,2023-06-01 00:10:11,2023-06-01 00:26:19,1,1.63,1,162,68,1,15.6,1.0,0.5,4.12,0.0,1.0,24.72,2.5,0.0,16.133333333333333,15.16564417177914,20.9,64.4,0.23
2,2023-06-01 00:08:47,2023-06-01 00:56:03,1,23.51,2,132,50,1,70.0,0.0,0.5,16.46,6.55,1.0,98.76,2.5,1.75,47.266666666666666,4.200765631646108,20.9,64.4,0.23
1,2023-06-01 00:00:12,2023-06-01 00:08:24,1,1.5,1,142,162,1,10.0,3.5,0.5,3.0,0.0,1.0,18.0,2.5,0.0,8.2,12.0,20.9,64.4,0.23
2,2023-06-01 00:34:15,2023-06-01 00:39:23,2,1.51,1,234,125,1,7.9,1.0,0.5,2.58,0.0,1.0,15.48,2.5,0.0,5.133333333333334,10.251655629139073,20.9,64.4,0.23


In [10]:
scripts.clean_base.write_data(yellow_with_weather_df, "../data/curated/yellow/yellow_weather.parquet")

In [11]:
scripts.plot_data.plot_correlation_heatmap(yellow_with_weather_df, scripts.clean_yellow.COLUMNS + weather_attrs, "yellow", "../plots/yellow/correlation/", "with_Weather")

24/08/27 18:58:45 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


../plots/yellow/correlation/with_Weather.png


In [12]:
uber_df = spark.read.parquet("../data/curated/uber/uber.parquet")
uber_df = uber_df.withColumn("datetime", to_date(col("pickup_datetime")))
uber_with_weather_df = uber_df.join(weather_df, on="datetime", how="inner")
uber_with_weather_df = uber_with_weather_df.drop("datetime")
uber_with_weather_df

dispatching_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_distance,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,total_amount,waiting_time,fare_per_miles,temp,humidity,precip
B03404,2023-06-22 13:15:39,2023-06-22 13:16:29,2023-06-22 13:16:43,2023-06-22 13:29:42,230,142,1.6,12.983333333333333,17.19,0.0,0.52,1.53,2.75,0.0,0.0,9.56,21.990000000000002,0.8333333333333334,13.74375,18.6,71.9,0.005
B03404,2023-06-22 13:49:36,2023-06-22 13:50:10,2023-06-22 13:50:27,2023-06-22 13:56:11,132,132,1.47,5.733333333333333,7.27,0.0,0.29,0.87,0.0,2.5,0.0,5.39,10.93,0.5666666666666667,7.4353741496598635,18.6,71.9,0.005
B03404,2023-06-22 13:33:22,2023-06-22 13:42:33,2023-06-22 13:43:06,2023-06-22 14:24:52,42,161,6.81,41.766666666666666,31.97,0.0,0.96,2.84,0.75,0.0,0.0,26.74,36.519999999999996,9.183333333333334,5.3627019089574155,18.6,71.9,0.005
B03404,2023-06-22 13:43:29,2023-06-22 13:45:53,2023-06-22 13:46:56,2023-06-22 15:24:37,211,98,19.93,97.68333333333334,152.01,6.55,4.76,14.07,2.75,0.0,34.56,107.72,214.7,2.4,10.772704465629703,18.6,71.9,0.005
B03404,2023-06-22 12:56:43,2023-06-22 13:01:22,2023-06-22 13:02:17,2023-06-22 13:18:00,94,235,1.83,15.716666666666667,10.5,0.0,0.32,0.93,0.0,0.0,0.0,11.29,11.75,4.65,6.420765027322404,18.6,71.9,0.005
B03404,2023-06-22 13:15:41,2023-06-22 13:17:27,2023-06-22 13:17:44,2023-06-22 13:38:07,107,246,1.71,20.383333333333333,17.17,0.0,0.52,1.52,2.75,0.0,0.0,17.13,21.96,1.7666666666666666,12.842105263157896,18.6,71.9,0.005
B03404,2023-06-22 13:20:08,2023-06-22 13:27:17,2023-06-22 13:27:17,2023-06-22 13:36:00,116,127,3.36,8.716666666666667,13.36,0.0,0.4,1.19,0.0,0.0,0.0,9.54,14.95,7.15,4.449404761904762,18.6,71.9,0.005
B03404,2023-06-22 13:55:42,2023-06-22 13:56:56,2023-06-22 13:57:29,2023-06-22 14:08:23,166,168,2.12,10.9,13.35,0.0,0.4,1.18,0.0,0.0,0.0,8.94,14.93,1.2333333333333334,7.042452830188679,18.6,71.9,0.005
B03404,2023-06-22 13:39:47,2023-06-22 13:44:00,2023-06-22 13:44:02,2023-06-22 14:01:54,243,42,5.73,17.866666666666667,20.48,0.0,0.61,1.82,0.0,0.0,0.0,17.61,22.91,4.216666666666667,3.9982547993019195,18.6,71.9,0.005
B03404,2023-06-22 13:53:25,2023-06-22 13:53:58,2023-06-22 13:55:58,2023-06-22 14:37:53,237,211,4.14,41.916666666666664,45.41,0.0,1.36,4.03,2.75,0.0,0.0,29.78,53.55,0.55,12.934782608695652,18.6,71.9,0.005


In [13]:
scripts.clean_base.write_data(uber_with_weather_df, "../data/curated/uber/uber_weather.parquet")

In [14]:
scripts.plot_data.plot_correlation_heatmap(uber_with_weather_df, scripts.clean_high_volume.COLUMNS + weather_attrs, "uber", "../plots/uber/correlation/", "with_Weather")

../plots/uber/correlation/with_Weather.png
